![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
// We need to load assemblies at the start in their own cell
#load "../Initialize.csx"
#load "../QuantConnect.csx"

using System;
using System.Linq;
using System.Collections.Generic;

using QuantConnect;
using QuantConnect.Data.Market;
using QuantConnect.Research;

In [ ]:
using System;
using System.Linq;
using System.Collections.Generic;
using Newtonsoft.Json;

// Helper: round decimal numbers to 2 places
decimal Round2(decimal value) => Math.Round(value, 2);

// Weekly bar DTO (camelCase, date strings)
public class WeeklyBarRecord
{
    public string startDate { get; set; }
    public string endDate { get; set; }
    public decimal open { get; set; }
    public decimal high { get; set; }
    public decimal low { get; set; }
    public decimal close { get; set; }
    public decimal volume { get; set; }
    public int tradingDays { get; set; }
    public decimal openToHigh { get; set; }
    public decimal openToLow { get; set; }
    public decimal openToClose { get; set; }
    public decimal lowToHigh { get; set; }
}

// Create QuantBook
var qb = new QuantBook();

// ----------------------
// Parameters
// ----------------------
const int MinTradingDays = 4;
const string TickerSymbol = "QQQ";

var HistoryStartDate = new DateTime(1998, 1, 1);
var HistoryEndDate   = new DateTime(2025, 12, 1);

// ----------------------
// Load daily price history
// ----------------------
var equity = qb.AddEquity(TickerSymbol, Resolution.Daily);
equity.SetDataNormalizationMode(DataNormalizationMode.SplitAdjusted);

var dailyHistory = qb.History<TradeBar>(
    equity.Symbol,
    HistoryStartDate,
    HistoryEndDate,
    Resolution.Daily
)
.OrderBy(b => b.EndTime)
.ToList();

// ----------------------
// Group into weekly bars
// ----------------------
var weeklyBars = new List<TradeBar>();
var weeklyTradingDates = new Dictionary<DateTime, List<DateTime>>();

var groupedByWeek = dailyHistory
    .GroupBy(bar =>
    {
        int dayOfWeek = (int)bar.Time.DayOfWeek;   // Sunday = 0
        int mondayOffset = dayOfWeek == 0 ? -6 : 1 - dayOfWeek;
        return bar.Time.Date.AddDays(mondayOffset);
    })
    .OrderBy(g => g.Key);

foreach (var group in groupedByWeek)
{
    var weekBars = group.OrderBy(x => x.Time).ToList();

    if (weekBars.Count < MinTradingDays)
        continue;

    var first = weekBars.First();
    var last  = weekBars.Last();

    var weekly = new TradeBar
    {
        Time = first.Time,
        Symbol = first.Symbol,
        Open   = first.Open,
        High   = weekBars.Max(b => b.High),
        Low    = weekBars.Min(b => b.Low),
        Close  = last.Close,
        Volume = weekBars.Sum(b => b.Volume)
    };

    weeklyBars.Add(weekly);
    weeklyTradingDates[weekly.Time] = weekBars.Select(b => b.Time.Date).ToList();
}

// ----------------------
// Build weekly JSON objects
// ----------------------
var weeklyRecords = new List<WeeklyBarRecord>();

foreach (var bar in weeklyBars)
{
    decimal open  = bar.Open;
    decimal high  = bar.High;
    decimal low   = bar.Low;
    decimal close = bar.Close;

    decimal pctOpenToHigh  = open != 0 ? (high - open)  / open * 100m : 0m;
    decimal pctOpenToLow   = open != 0 ? (low  - open)  / open * 100m : 0m;
    decimal pctOpenToClose = open != 0 ? (close - open) / open * 100m : 0m;
    decimal pctLowToHigh   = low  != 0 ? (high - low)   / low  * 100m : 0m;

    weeklyTradingDates.TryGetValue(bar.Time, out var dates);
    var endDate = dates?.Last() ?? bar.Time.Date;

    weeklyRecords.Add(new WeeklyBarRecord
    {
        startDate    = bar.Time.Date.ToString("yyyy-MM-dd"),
        endDate      = endDate.ToString("yyyy-MM-dd"),
        open         = Round2(open),
        high         = Round2(high),
        low          = Round2(low),
        close        = Round2(close),
        volume       = Round2(bar.Volume),
        tradingDays  = dates?.Count ?? 0,
        openToHigh   = Round2(pctOpenToHigh),
        openToLow    = Round2(pctOpenToLow),
        openToClose  = Round2(pctOpenToClose),
        lowToHigh    = Round2(pctLowToHigh)
    });
}

// ----------------------
// Build final JSON object
// ----------------------
var finalObject = new
{
    ticker = TickerSymbol,
    analysisStartDate = HistoryStartDate.ToString("yyyy-MM-dd"),
    analysisEndDate = HistoryEndDate.ToString("yyyy-MM-dd"),
    minTradingDayWeekThreshold = MinTradingDays,
    weeklyData = weeklyRecords
};

// ----------------------
// Output JSON
// ----------------------
JsonConvert.SerializeObject(finalObject, Formatting.Indented)
